In [1]:
import pandas as pd
import string
from sklearn.preprocessing import MinMaxScaler

In [2]:
df = pd.read_csv("data.csv")

In [3]:
del(df['Unnamed: 0'])

In [4]:
for i in range(len(df['file_name'])):
    df["file_name"][i] = df["file_name"][i].split('.fpg',1)[0]

In [5]:
print(df["file_name"][38])

0001f13


In [6]:
df.columns = ['File Name','X','Y','Pressure','Azumith','Pen Elevation']

In [7]:
colnames = list(df.columns)
filenames = list(df['File Name'])
def clean(t):
  bad = ['.', '\n', '[',']'] 
  t = ''.join(i for i in t if not i in bad) 
  t = t.split(' ')
  t = list(filter(None, t)) 
  t = [float(i) for i in t]
  return(t)
  
df = df.set_index([colnames[0]])
for i in range(1,6):
  df[colnames[i]] = df[colnames[i]].map(clean)

In [8]:
data = {}
auth = {}
scaler = MinMaxScaler()
for timeseries in filenames:
  data[timeseries] = pd.DataFrame()
  auth[timeseries] = 0 if timeseries[4]=='f' else 1 
  for j in range(1,6):
    data[timeseries][colnames[j]] = df[colnames[j]][timeseries]
  offset = data[timeseries].min(axis = 0)[0:2]
  data[timeseries]['X'] -= offset[0]
  data[timeseries]['Y'] -= offset[1]
  data[timeseries][colnames[1:3]] = pd.DataFrame(scaler.fit_transform(data[timeseries][colnames[1:3]]), columns=colnames[1:3])

In [13]:
print(data["0001f13"])

            X         Y  Pressure  Azumith  Pen Elevation
0    0.014974  0.263484      24.0    152.0           57.0
1    0.006070  0.180371      29.0    151.0           58.0
2    0.000000  0.189213      54.0    151.0           58.0
3    0.000000  0.176835     103.0    150.0           58.0
4    0.000000  0.169761     142.0    150.0           58.0
5    0.005666  0.169761     181.0    150.0           58.0
6    0.010117  0.169761     225.0    150.0           58.0
7    0.014164  0.177719     264.0    150.0           58.0
8    0.025900  0.194518     302.0    150.0           58.0
9    0.042898  0.231653     334.0    152.0           59.0
10   0.061109  0.281167     399.0    151.0           60.0
11   0.079320  0.338638     412.0    151.0           60.0
12   0.095103  0.410256     414.0    151.0           60.0
13   0.108863  0.503095     448.0    151.0           60.0
14   0.124241  0.610080     442.0    152.0           59.0
15   0.137596  0.706454     447.0    150.0           58.0
16   0.149737 

In [17]:
# Creating separate label dictionary to store labels(1 or 0 for true or false signature respectively) 
# with the format {file_name: label}
label = {}
for key in data:
    if(key.find('f') != -1):
        label[key] = 0
    elif(key.find('v') != -1):
        label[key] = 1
print(label["0001v13"])  #should print 1
print(label["0001f13"])  #should print 0

1
0
